                                            DATA MAPPER                                                 

1st Version -

-> Tackling how to convert categorical data to numerical so algorithm understands it,

-> find the algorithm which helps in storing data in numerical form without loosing the meaning of the 
sentence. 

-> Utilize cosine similarity to get the matches from LHS to RHS.

-> methods to visualize results

In [2]:
!pip install torch
!pip install sentence-transformers
!pip install pandas
!pip install matplotlib
!pip install --upgrade transformers sentence-transformers


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: C:\Users\Krithika.Patali\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: C:\Users\Krithika.Patali\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: C:\Users\Krithika.Patali\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: C:\Users\Krithika.Patali\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: C:\Users\Krithika.Patali\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import matplotlib.pyplot as plt
import numpy as np

In [4]:
#Load csv files
#df1 = pd.read_csv(r"C:\Users\Yasvanth.Pamidi\OneDrive - ENCORA\Desktop\DataMap\metadata files\LHS.csv") #path to file1
#df2 = pd.read_csv(r"C:\Users\Yasvanth.Pamidi\OneDrive - ENCORA\Desktop\DataMap\metadata files\RHS.csv") #path to file2
#df1 = pd.read_csv("./LHS 2.csv") #path to file1
#df2 = pd.read_csv("./RHS 3.csv") #path to file2
df1=pd.read_csv(r"C:\Users\Krithika.Patali\DataMapper\LHS_3.csv")
df2=pd.read_csv(r"C:\Users\Krithika.Patali\Downloads\RHS 3.csv")

#define columns to compare by converting columns in to string format

column1 = 'Description' #column name in df1
df1[column1] = df1[column1].astype(str)

column2 = 'Description' #column name in df2
df2[column2] = df2[column2].astype(str)

In [5]:

# Load a pre-trained model
model = SentenceTransformer('sentence-transformers/paraphrase-mpnet-base-v2')

In [6]:
#Generate Embeddings for each sentense in the selected columns

embeddings1 = model.encode(df1[column1].to_list(), show_progress_bar=True,convert_to_tensor=True)
embeddings2 = model.encode(df2[column2].to_list(), show_progress_bar=True,convert_to_tensor=True)

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/15 [00:00<?, ?it/s]

In [7]:
#compute the cosine_similarity_matrix

similarity_matrix = util.cos_sim(embeddings1,embeddings2).cpu().numpy()

In [9]:
# Select a specific sentence from embeddings1, e.g., the first sentence (index 0)
selected_index = 128
similarities = similarity_matrix[selected_index]  # Get similarity scores for the selected sentence

# Find the indices of the top 3 most similar sentences in embeddings2
top_3_indices = np.argsort(similarities)[-3:][::-1]  # Sort and get indices in descending order

#print the actual sentence
print('Actual Sentence: '+df1[column1][selected_index])

# Print the top 3 indices and their similarity scores
for idx in top_3_indices:
    print(f"Sentence index in embeddings2: {idx}, Similarity: {similarities[idx]}")

# Example: If you want to retrieve the actual sentences, ensure you have access to the original sentences.
# embeddings2_sentences is assumed to be the list of sentences corresponding to embeddings2
top_3_sentences = [df2[column2][idx] for idx in top_3_indices]
print("Top 3 most similar sentences:", top_3_sentences)

Actual Sentence: Assortment List Type
Sentence index in embeddings2: 83, Similarity: 0.3955927789211273
Sentence index in embeddings2: 259, Similarity: 0.3921678066253662
Sentence index in embeddings2: 2, Similarity: 0.3866003453731537
Top 3 most similar sentences: ['Denotes the type of the product.\nValues:\n1. FPS\n2. FPC \n3. OG\n4. OO \n5. Item Only\n6. Shipper \n7. Ingredient ', 'Category in which the item is picked & grouped into Totes by Penske . Also used by stores to determine what is in the totes so that they can go to the correct location in store', 'Variety of item, as defined by the MCoE at sub-category level. Attribute managed by MCoE']


In [10]:
import json

# Function to retrieve top-N most similar sentences for all sentences in embeddings1
def retrieve_top_similar_sentences_json(num_sentences, top_n):
    results = []  # List to hold all results

    for selected_index in range(num_sentences):
        # Get similarity scores for the selected sentence
        similarities = similarity_matrix[selected_index]

        # Find the indices of the top N most similar sentences in embeddings2
        top_indices = np.argsort(similarities)[-top_n:][::-1]  # Sort in descending order

        # Construct matches
        matches = []
        for rank, idx in enumerate(top_indices, start=1):
            matches.append({
                f"rhs_field_desc_{rank}": df2[column2][idx],
                "rhs_field_score": float(similarities[idx])  # Convert to native Python float
            })

        # Construct the result for the current LHS sentence
        result = {
          #  f"lhs_field_{selected_index}": df1[column1][selected_index],
            f"lhs_desc_{selected_index}": df1[column1][selected_index],
            "matches": matches
        }

        # Append to results
        results.append(result)

    # Format as JSON
    final_output = {"results": results}
    return final_output

# User inputs: number of sentences to process and top-N similarities to retrieve
num_sentences_to_process = int(input("Enter the number of sentences from LHS to process: ") or len(df1))
top_n = int(input("Enter the number of top similar sentences to retrieve (N): ") or 3)

# Retrieve results in the specified JSON format
output_json = retrieve_top_similar_sentences_json(num_sentences_to_process, top_n)

# Print the output as formatted JSON
print(json.dumps(output_json, indent=2))



{
  "results": [
    {
      "lhs_desc_0": "Client",
      "matches": [
        {
          "rhs_field_desc_1": "short name of the option set ; customer facing",
          "rhs_field_score": 0.44968175888061523
        },
        {
          "rhs_field_desc_2": "identifies if the product is offered thru Web, Catering lite , retil etc",
          "rhs_field_score": 0.3524893820285797
        },
        {
          "rhs_field_desc_3": "The WAWA employee email acting as the primary contact for the item",
          "rhs_field_score": 0.3440648913383484
        }
      ]
    },
    {
      "lhs_desc_1": "Material Number",
      "matches": [
        {
          "rhs_field_desc_1": "Equipment model number from the manufacturer",
          "rhs_field_score": 0.5605772137641907
        },
        {
          "rhs_field_desc_2": "The quantity of each component item in a shipper display",
          "rhs_field_score": 0.5381794571876526
        },
        {
          "rhs_field_desc_3": "Number of

                                                        version 2                                                       

Version 2: 

-> Added sample datatype check on top of similarity score.

-> Enhanced the model

-> Added new representation with more information

In [11]:
!pip install tqdm
!pip install ipywidgets


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: C:\Users\Krithika.Patali\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: C:\Users\Krithika.Patali\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [12]:
import numpy as np
import torch
from tqdm import tqdm

In [13]:
#function to encode sentences in batches
def batch_encode(column, batch_size, model):
    embeddings = []
    column = column.tolist()
    with torch.no_grad():
        for i in tqdm(range(0, len(column), batch_size), desc = "Encoding Batches"):
            batch = column[i:i+batch_size]
            batch_embeddings = model.encode(batch, convert_to_tensor= True,show_progress_bar=True)
            embeddings.append(batch_embeddings)
    return torch.cat(embeddings, dim=0)


embeddings1 = batch_encode(df1[column1], batch_size=128, model=model)
embeddings2 = batch_encode(df2[column2], batch_size=128, model=model)

Encoding Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Encoding Batches:  50%|█████     | 1/2 [00:01<00:01,  1.77s/it]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Encoding Batches:   0%|          | 0/4 [00:00<?, ?it/s]

...

Encoding Batches:  50%|█████     | 2/4 [00:06<00:06,  3.46s/it]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Encoding Batches:  75%|███████▌  | 3/4 [00:09<00:03,  3.13s/it]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Encoding Batches: 100%|██████████| 4/4 [00:11<00:00,  2.85s/it]


In [14]:
#compute the cosine_similarity_matrix here output will be in range of -1 to 1

similarity_matrix = util.cos_sim(embeddings1,embeddings2).cpu().numpy()

In [15]:
# normalizing the similarity matrix as the out values to range from 0 to 1 
normalized_similarity_matrix = (similarity_matrix + 1) / 2


In [16]:
# To verify the Data Types as we have data belonging to different databases 


type_mapping = {
    "CHAR": ["VARCHAR"],
    "NUMC": ["INTEGER", "BIGINT", "NUMERIC"],
    "DATS": ["DATE"],
    "TIMS": ["TIME"],
    "DEC": ["NUMERIC", "FLOAT"],
    "INT4": ["INTEGER", "BIGINT"]
}


ONE TO MANY - Below code gets the topN similarities from RHS for particular index on LHS and shows if data types are compatible or not

In [17]:
import json
import numpy as np
import pandas as pd

# Function to retrieve top-N most similar sentences with data type validation
def retrieve_top_similar_sentences_json(selected_index, top_n, similarity_matrix, df1, df2, column1, column2, type_mapping):
    # Initialize variables
    similarities = similarity_matrix[selected_index]
    top_indices = np.argsort(similarities)[-top_n:][::-1]  # Top-N indices sorted in descending order
    
    lhs_field = df1.loc[selected_index, 'Field Name']  # LHS field name
    lhs_desc = df1.loc[selected_index, column1]        # LHS description
    lhs_type = df1.loc[selected_index, 'Data_Type']    # LHS data type
    
    # List to store all matches
    matches = [
        {
            "rank": rank + 1,
            "rhs_field_score": float(similarities[idx]),
            "rhs_index": int(df2.index[idx]),
            "rhs_field_name": df2.loc[idx, 'Attribute'],   # RHS attribute name
            "rhs_field_desc": df2.loc[idx, column2],       # RHS description
            "rhs_data_type": df2.loc[idx, 'Data_Type'],    # RHS data type
            "compatibility": "Compatible" if df2.loc[idx, 'Data_Type'] in type_mapping.get(lhs_type, []) else "Incompatible"
        }
        for rank, idx in enumerate(top_indices)  # Loop through top indices
    ]

    # Construct result dictionary
    result = {
                f"lhs_field_index": selected_index,
                f"lhs_field_name": lhs_field,
                f"lhs_field_description": lhs_desc,
                f"lhs_field_data_type": lhs_type,
                "matches": matches
            }

    # Return final JSON output
    return {"results": [result]}


# User inputs
enter_index = int(input("Enter the index from LHS to process: "))
top_n = int(input("Enter the number of top similar sentences to retrieve (N): ") or 3)

# Retrieve and display results
output_json = retrieve_top_similar_sentences_json(
    enter_index, top_n, normalized_similarity_matrix, df1, df2, column1, column2, type_mapping
)

print(json.dumps(output_json, indent=2))


{
  "results": [
    {
      "lhs_field_index": 3,
      "lhs_field_name": "ERNAM",
      "lhs_field_description": "Name of Person who Created the Object",
      "lhs_field_data_type": "CHAR",
      "matches": [
        {
          "rank": 1,
          "rhs_field_score": 0.7057956457138062,
          "rhs_index": 63,
          "rhs_field_name": "Item Name From Vendor",
          "rhs_field_desc": "name of the item as represented by the seller / vendor",
          "rhs_data_type": "VARCHAR",
          "compatibility": "Compatible"
        },
        {
          "rank": 2,
          "rhs_field_score": 0.6697574853897095,
          "rhs_index": 178,
          "rhs_field_name": "Brand name",
          "rhs_field_desc": "Brand name of product",
          "rhs_data_type": "VARCHAR",
          "compatibility": "Compatible"
        },
        {
          "rank": 3,
          "rhs_field_score": 0.6614560484886169,
          "rhs_index": 85,
          "rhs_field_name": "Product Logo",
          

ONE TO MANY - Below code gets the topN similarities from RHS for particular index on LHS and shows if data types are compatible

In [18]:
import json
import numpy as np
import pandas as pd

# Function to retrieve top-N most similar sentences with data type validation
def retrieve_top_similar_sentences_json(selected_index, top_n, similarity_matrix, df1, df2, column1, column2, type_mapping):
    # Initialize variables
    similarities = similarity_matrix[selected_index]
    top_indices = np.argsort(similarities)[-top_n:][::-1]  # Top-N indices sorted in descending order
    
    lhs_field = df1.loc[selected_index, 'Field Name']  # LHS field name
    lhs_desc = df1.loc[selected_index, column1]        # LHS description
    lhs_type = df1.loc[selected_index, 'Data_Type']    # LHS data type
    
    # List to store all matches
    matches = [
        {
            "rank": rank + 1,
            "rhs_field_score": float(similarities[idx]),
            "rhs_index": int(df2.index[idx]),
            "rhs_field_name": df2.loc[idx, 'Attribute'],   # RHS attribute name
            "rhs_field_desc": df2.loc[idx, column2],       # RHS description
            "rhs_data_type": df2.loc[idx, 'Data_Type'],    # RHS data type
            "compatibility": "Compatible" if df2.loc[idx, 'Data_Type'] in type_mapping.get(lhs_type, []) else "Incompatible"
        }
        for rank, idx in enumerate(top_indices)  # Loop through top indices
        if (rhs_type := df2.loc[idx, 'Data_Type']) in type_mapping.get(lhs_type, [])

    ]

    # Construct result dictionary
    result = {
                f"lhs_field_index": selected_index,
                f"lhs_field_name": lhs_field,
                f"lhs_field_description": lhs_desc,
                f"lhs_field_data_type": lhs_type,
                "matches": matches
            }

    # Return final JSON output
    return {"results": [result]}


# User inputs
enter_index = int(input("Enter the index from LHS to process: "))
top_n = int(input("Enter the number of top similar sentences to retrieve (N): ") or 3)

# Retrieve and display results
output_json = retrieve_top_similar_sentences_json(
    enter_index, top_n, normalized_similarity_matrix, df1, df2, column1, column2, type_mapping
)

print(json.dumps(output_json, indent=2))


{
  "results": [
    {
      "lhs_field_index": 45,
      "lhs_field_name": "BWVOR",
      "lhs_field_description": "Procurement rule",
      "lhs_field_data_type": "CHAR",
      "matches": []
    }
  ]
}


                                                    Version 3                                                           

Version 3-

-> Updated the data type check with extension version such that we can add any database in future.

-> Uodated the code with more information showing in results

-> Updated such that it shows only similarities based on datatype compatibility = true

In [19]:
import os
import json


In [34]:
#mappings_dir = r"C:\Users\Yasvanth.Pamidi\OneDrive - ENCORA\Desktop\VSC\DataMapper\versions\mappings"
mappings_dir = r"C:/Users/Krithika.Patali/DataMapper/Mappings"

load_mappings : 

-> Initializes an empty dictionary to store database mappings

-> Checks if the specified directory exists. If not, raises a FileNotFoundError

-> Iterates through files in the directory: Selects only .json files, excluding compatibilities.json. Extracts the database name from the filename. Opens the JSON file in read mode, reads its contents using json.load, and stores it in the mappings dictionary under the respective database name.

-> Checks if no mappings were loaded and raises a ValueError.

-> Returns the mappings dictionary containing database-to-data type mappings.

In [29]:
def load_mappings(mappings_dir):
    mappings = {} # Initializes an empty dictionary to store database mappings

    print(f"Loading mappings from directory: {mappings_dir}")

    if not os.path.exists(mappings_dir):
        raise FileNotFoundError(f"Mappings directory '{mappings_dir}' does not exist.") #Checks if the specified directory exists. If not, raises a FileNotFoundError
    for filename in os.listdir(mappings_dir):
        if filename.endswith(".json") and filename != "compatibilities.json":
            database_name = os.path.splitext(filename)[0]

            print(f"Processing file: {filename} as database: {database_name}")
            
            with open(os.path.join(mappings_dir, filename), "r") as file:
                mappings[database_name] = json.load(file) #Iterates through files in the directory: Selects only .json files, excluding compatibilities.json. Extracts the database name from the filename. Reads the JSON file and loads its content into the mappings dictionary.

                print(f"Loaded mapping for {database_name}: {mappings[database_name]}")
    
    if not mappings:
        raise ValueError(f"No mapping files found in '{mappings_dir}'.") #Checks if no mappings were loaded and raises a ValueError.
    
    print("Mappings loaded successfully:", mappings)
    return mappings #Returns the mappings dictionary containing database-to-data type mappings.

load_compatibilities: Reads the compatibility JSON file and loads it into a Python dictionary.

In [30]:
def load_compatibilities(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

normalize_data_type:

-> Normalizes the data type of a field by converting it to lowercase

-> Prepares a list to store databases where a match is found.

-> Iterates through each database in the mappings.

-> Converts all keys in the mapping to lowercase to ensure case-insensitive lookup.

-> If a match for rhs_type_lower is found, it: Updates normalized_type. Adds the database to matching_databases.

-> If matches are found, returns the normalized type and list of matching databases. Otherwise, returns the original type (converted to uppercase) and None.

In [35]:
def normalize_data_type(rhs_type,database_context, mappings):
    
    print(f"\nNormalizing RHS Type: {rhs_type} using mappings")
    
    rhs_type_lower = rhs_type.lower()

    print(f"Lowercase RHS Type: {rhs_type_lower}")

    matching_databases = []
    normalized_type = None #Normalizes the data type of a field by converting it to lowercase, Prepares a list to store databases where a match is found.

    for db_name, db_mapping in mappings.items():

        print(f"Checking database: {db_name}")

        normalized = {k.lower(): v for k, v in db_mapping.items()}.get(rhs_type_lower)
        if normalized:
            normalized_type = normalized
            matching_databases.append(db_name)

            print(f"Match found in {db_name}: Normalized Type -> {normalized_type}")

    if matching_databases:
        
        print(f"Final Normalized Type: {normalized_type}, Matching Databases: {matching_databases}")

        return normalized_type, matching_databases
    
    
    print(f"No matches found. Returning RHS Type in uppercase: {rhs_type.upper()}")

    return rhs_type.upper(), None

are_compatible: 

Purpose: Checks if two data types (lhs_type and rhs_type) are compatible.

-> Converts both types to uppercase for consistency.

-> Checks: 1:If the two types are identical. 2:If rhs_type exists in the compatibility list of lhs_type.

In [36]:
def are_compatible(lhs_type, rhs_type, compatibilities):
    lhs_type = lhs_type.upper()
    rhs_type = rhs_type.upper()
    return lhs_type == rhs_type or rhs_type in compatibilities.get(lhs_type, [])

In [37]:
def retrieve_top_similar_sentences_json(selected_index, top_n, normalized_similarity_matrix, df1, df2, column1, column2, type_mapping, compatibilities, filter_compatible=True):
    if normalized_similarity_matrix.shape != (len(df1), len(df2)):

        print(f"Error: Similarity matrix shape {normalized_similarity_matrix.shape} does not match dataframes: LHS {len(df1)}, RHS {len(df2)}")

        raise ValueError("The similarity matrix dimensions must match the LHS and RHS datasets.")
    
    print("Input validation passed. Proceeding...")
    
    similarities = normalized_similarity_matrix[selected_index]
    matches = []
    for idx in range(len(similarities)):
        rhs_field = df2.loc[idx, 'Attribute']
        rhs_desc = df2.loc[idx, column2]
        rhs_type = df2.loc[idx, 'Data_Type']

        print(f"\nProcessing RHS Index {idx}:")
        print(f"  Field: {rhs_field}, Description: {rhs_desc}, Type: {rhs_type}")

        database_context = df2.loc[idx, 'Database'] if 'Database' in df2.columns else "default_database"
        normalized_rhs_type, database_context = normalize_data_type(rhs_type, None, type_mapping)

        print(f"  Normalized RHS Type: {normalized_rhs_type}, Database Context: {database_context}")

        lhs_type = df1.loc[selected_index, 'Data_Type'].upper()
        normalized_lhs_type, _ = normalize_data_type(lhs_type, None, type_mapping)

        print(f"  LHS Type: {lhs_type}, Normalized LHS Type: {normalized_lhs_type}")

        normalized_rhs_type = normalized_rhs_type.upper()
        is_compatible = are_compatible(lhs_type, normalized_rhs_type, compatibilities)
        if is_compatible :
            matches.append({
                "rank": None,  # Placeholder to set rank later
                "similarity_score": float(similarities[idx]),
                "rhs_index": int(df2.index[idx]),
                "rhs_field_name": rhs_field,
                "rhs_field_desc": rhs_desc,
                "rhs_data_type": rhs_type,
                "normalized_rhs_type": normalized_rhs_type,
                "database_name": database_context if database_context else "Unknown",
                "compatibility": "Compatible",
            })
    matches = sorted(matches, key=lambda x: x["similarity_score"], reverse=True)

    print("\nSorted Matches by Similarity Score:")

    matches = matches[:top_n]
    for rank, match in enumerate(matches, start=1):
        match["rank"] = rank
    if not matches:
        matches.append({"message": "No compatible matches found."})
    lhs_field = df1.loc[selected_index,'Field Name']
    lhs_desc = df1.loc[selected_index, column1]
    lhs_type = df1.loc[selected_index, 'Data_Type']
    result = {
        "lhs_field_index": selected_index,
        "lhs_field_name": lhs_field,
        "lhs_field_description": lhs_desc,
        "lhs_field_data_type": lhs_type,
        "normalized_lhs_type": normalized_lhs_type,
        "matches": matches
    }
    return {"results": [result]}

type_mapping = load_mappings(mappings_dir)
compatibilities_file = os.path.join(mappings_dir, 'compatibilities.json')
compatibilities = load_compatibilities(compatibilities_file)

enter_index = int(input("Enter the index from LHS to process: "))
top_n = int(input("Enter the number of top similar sentences to retrieve (N): ") or 3)

check_length = True  #Adding Length

output_json = retrieve_top_similar_sentences_json(
    enter_index, top_n, normalized_similarity_matrix, df1, df2, "Description", "Description", type_mapping, compatibilities, filter_compatible=True
)
print(json.dumps(output_json, indent=2))

Loading mappings from directory: C:/Users/Krithika.Patali/DataMapper/Mappings
Processing file: pgsql_sql_2023_map.json as database: pgsql_sql_2023_map
Loaded mapping for pgsql_sql_2023_map: {'TEXT': 'VARCHAR', 'INTEGER': 'INT', 'VARCHAR': 'VARCHAR', 'FLOAT': 'FLOAT', 'BOOLEAN': 'BIT', 'BIGINT': 'BIGINT'}
Processing file: saphana_sql2023_map.json as database: saphana_sql2023_map
Loaded mapping for saphana_sql2023_map: {'NVARCHAR': 'NVARCHAR', 'VARCHAR': 'VARCHAR', 'CHAR': 'CHAR', 'DATE': 'DATE', 'TIME': 'TIME', 'SECONDDATE': 'DATETIME2', 'TIMESTAMP': 'TIMESTAMP', 'DECIMAL': 'DECIMAL', 'SMALLINT': 'SMALLINT', 'INTEGER': 'INT', 'BIGINT': 'BIGINT', 'FLOAT': 'FLOAT', 'REAL': 'REAL', 'DOUBLE': 'FLOAT', 'CLOB': 'VARCHAR(max)', 'NCLOB': 'NVARCHAR(max)', 'BLOB': 'VARBINARY(max)', 'BOOLEAN': 'BIT', 'ST_GEOMETRY': 'JSON', 'ST_POINT': 'JSON', 'JSON': 'JSON', 'GUID': 'UNIQUEIDENTIFIER', 'TINYINT': 'SMALLINT'}
Processing file: sapmara_sql2023_map.json as database: sapmara_sql2023_map
Loaded mapping 

Input validation passed. Proceeding...

Processing RHS Index 0:
  Field: Shipper Contents Name, Description: Items that make up the shipper display  (i.e. M&Ms 3 flavor shipper display), Type: TEXT

Normalizing RHS Type: TEXT using mappings
Lowercase RHS Type: text
Checking database: pgsql_sql_2023_map
Match found in pgsql_sql_2023_map: Normalized Type -> VARCHAR
Checking database: saphana_sql2023_map
Checking database: sapmara_sql2023_map
Checking database: sap_sql2023_map
Final Normalized Type: VARCHAR, Matching Databases: ['pgsql_sql_2023_map']
  Normalized RHS Type: VARCHAR, Database Context: ['pgsql_sql_2023_map']

Normalizing RHS Type: CHAR using mappings
Lowercase RHS Type: char
Checking database: pgsql_sql_2023_map
Checking database: saphana_sql2023_map
Match found in saphana_sql2023_map: Normalized Type -> CHAR
Checking database: sapmara_sql2023_map
Match found in sapmara_sql2023_map: Normalized Type -> CHAR
Checking database: sap_sql2023_map
Match found in sap_sql2023_map: No

                                                    Version-4                                                               

Version 4-

->  Added length compatibility.
-> Added functionality for user to select similarity score threshold

normalize_length: 

Purpose: Converts a length to an integer.

Logic: If the length is invalid (e.g., non-numeric or None), it returns None.

In [38]:
def normalize_length(length):
    try:
        return int(length)
    except (ValueError, TypeError):
        return None

Purpose: Determines if two lengths are compatible.

Logic:

-> If check_length is False, compatibility check is skipped

-> Normalizes lengths to ensure valid numerical values.

-> Handles missing values by returning False.

-> checks for logic

In [40]:
def is_length_compatible(lhs_length, rhs_length, check_length=True):
    """
    Checks if the LHS length is compatible with the RHS length.

    - If `check_length` is False, it always returns True.
    - If lengths are not valid integers, it returns False.
    - Returns True if `lhs_length` is less than or equal to `rhs_length`.

    Args:
        lhs_length: The length of the column from the LHS dataset.
        rhs_length: The length of the column from the RHS dataset.
        check_length (bool): Flag to enable or disable length checking.

    Returns:
        bool: True if compatible, False otherwise.
    """
    if not check_length:
        return True
    
    lhs_length = normalize_length(lhs_length)
    rhs_length = normalize_length(rhs_length)

    print(f"LHS Length: {lhs_length}, RHS Length: {rhs_length}")
    
    if lhs_length is None or rhs_length is None:
        print("One of the lengths is invalid. Returning False.")
        return False
    
    if lhs_length <= rhs_length:  # Fixed missing colon
        return True  # Ensures proper logical flow
    
    return False  # Explicit return if the condition is not met


In [41]:
# Function to convert numpy types to native Python types
def convert_to_native_type(value):
    if isinstance(value, (np.integer, np.int64)):
        return int(value)
    elif isinstance(value, (np.floating, np.float64)):
        return float(value)
    elif isinstance(value, np.ndarray):
        return value.tolist()
    else:
        return value

updated main function for length compatible with similarity score threshould user input

In [42]:
def retrieve_top_similar_sentences_json(selected_index, top_n, normalized_similarity_matrix, df1, df2, column1, column2, type_mapping, compatibilities, filter_compatible=True):
    
    """
    Retrieves the top similar sentences based on a similarity matrix while checking compatibility.

    Args:
        selected_index (int): Index of the LHS column to compare.
        top_n (int): Number of top matches to return.
        normalized_similarity_matrix (np.array): Precomputed similarity scores.
        df1 (pd.DataFrame): LHS dataset.
        df2 (pd.DataFrame): RHS dataset.
        column1 (str): Column name for description in LHS.
        column2 (str): Column name for description in RHS.
        type_mapping (dict): Mapping of data types.
        compatibilities (dict): Compatibility mapping.
        similarity_threshold (float): Minimum similarity score for a match.
        filter_compatible (bool, optional): Whether to filter only compatible matches. Defaults to True.

    Returns:
        dict: JSON output with matched results.
    """

    if normalized_similarity_matrix.shape != (len(df1), len(df2)):

        print(f"Error: Similarity matrix shape {normalized_similarity_matrix.shape} does not match dataframes: LHS {len(df1)}, RHS {len(df2)}")
        raise ValueError("The similarity matrix dimensions must match the LHS and RHS datasets.")
    
    print("Input validation passed. Processing similarity matches...")

    similarities = normalized_similarity_matrix[selected_index]
    matches = []

    normalized_lhs_type = None

    for idx in range(len(similarities)):
        similarity_score = float(similarities[idx])  # Extract similarity score
        if similarity_score < similarity_threshold:
            continue  # Skip if below threshold

        rhs_field = df2.loc[idx, 'Attribute']
        rhs_desc = df2.loc[idx, column2]
        rhs_type = df2.loc[idx, 'Data_Type']
        rhs_length = df2.loc[idx, 'Length'] # RHS length

        print(f"\nProcessing RHS Index {idx}:")
        print(f"  Field: {rhs_field}, Description: {rhs_desc}, Type: {rhs_type}, Length: {rhs_length}")

        database_context = df2.loc[idx, 'Database'] if 'Database' in df2.columns else "default_database"
        normalized_rhs_type, database_context = normalize_data_type(rhs_type, None, type_mapping)

        print(f"  Normalized RHS Type: {normalized_rhs_type}, Database Context: {database_context}")

        lhs_type = df1.loc[selected_index, 'Data_Type'].upper()
        lhs_length = df1.loc[selected_index, 'Length']  # LHS length
        normalized_lhs_type, _ = normalize_data_type(lhs_type, None, type_mapping)

        print(f"  LHS Type: {lhs_type}, Normalized LHS Type: {normalized_lhs_type}")

        normalized_rhs_type = normalized_rhs_type.upper()
        is_compatible = are_compatible(lhs_type, normalized_rhs_type, compatibilities)

        # Normalize length before checking compatibility
        #lhs_length = normalize_length(lhs_length)
        #rhs_length = normalize_length(rhs_length)

        length_compatible = is_length_compatible(lhs_length, rhs_length, check_length=True)
        
        print(f"length_compatible: {length_compatible}")

        if is_compatible and length_compatible:
            matches.append({
                "rank": None,  # Placeholder to set rank later
                "similarity_score": similarity_score,
                "rhs_index": int(df2.index[idx]),
                "rhs_field_name": rhs_field,
                "rhs_field_desc": rhs_desc,
                "rhs_data_type": rhs_type,
                "normalized_rhs_type": normalized_rhs_type,
                "database_name": database_context if database_context else "Unknown",
                "rhs_length": convert_to_native_type(rhs_length),
                "compatibility": "Compatible"
                 
            })

    # Sort matches by similarity score (descending)
    matches = sorted(matches, key=lambda x: x["similarity_score"], reverse=True)[:top_n]

    print("\nSorted Matches by Similarity Score:")

    for rank, match in enumerate(matches, start=1):
        match["rank"] = rank

    if not matches:
        matches.append({"message": "No compatible matches found."})

    # Fetch LHS details safely
    lhs_field = df1.loc[selected_index,'Field Name']
    lhs_desc = df1.loc[selected_index, column1]
    lhs_type = df1.loc[selected_index, 'Data_Type']
    lhs_length = df1.loc[selected_index, 'Length']   

    result = {
        "lhs_field_index": selected_index,
        "lhs_field_name": lhs_field,
        "lhs_field_description": lhs_desc,
        "lhs_field_data_type": lhs_type,
        "normalized_lhs_type": normalized_lhs_type,
        "lhs_field_length": convert_to_native_type(lhs_length),  
        "matches": matches
    }

    return {"results": [result]}

# Load type mappings and compatibilities
type_mapping = load_mappings(mappings_dir)
compatibilities_file = os.path.join(mappings_dir, 'compatibilities.json')
compatibilities = load_compatibilities(compatibilities_file)


# User Input with Validations

# Validate LHS Index
while True:
    try:
        enter_index = int(input("Enter the index from LHS to process: "))
        if 0<= enter_index <len(df1): # Ensure it's within range
            break
        else:
            print(f"Error: Index out of range. Please enter a value between 0 and {len(df1) - 1}.")
    except ValueError:
        print("Invalid input. Please enter a valid integer.")


# Validate Top N
while True:
    try:
        top_n = int(input("Enter the number of top similar sentences to retrieve (N): ") or 3)
        if top_n > 0:
            break
        else:
            print("Error: Please enter a positive integer greater than 0.")
    except ValueError:
        print("Invalid input. Please enter a valid integer.")

# Validate Similarity Score Threshold
while True:
    try:
        similarity_threshold = float(input("Enter the similarity score threshold (0.0 - 1.0): "))
        if 0.0 <= similarity_threshold <= 1.0:
            break
        else:
            print("Error: Please enter a value between 0.0 and 1.0.")
    except ValueError:
        print("Invalid input. Please enter a numeric value between 0.0 and 1.0.")

output_json = retrieve_top_similar_sentences_json(
    enter_index, top_n, normalized_similarity_matrix, df1, df2, "Description", "Description", type_mapping, compatibilities,similarity_threshold)
print(json.dumps(output_json, indent=2))

Loading mappings from directory: C:/Users/Krithika.Patali/DataMapper/Mappings
Processing file: pgsql_sql_2023_map.json as database: pgsql_sql_2023_map
Loaded mapping for pgsql_sql_2023_map: {'TEXT': 'VARCHAR', 'INTEGER': 'INT', 'VARCHAR': 'VARCHAR', 'FLOAT': 'FLOAT', 'BOOLEAN': 'BIT', 'BIGINT': 'BIGINT'}
Processing file: saphana_sql2023_map.json as database: saphana_sql2023_map
Loaded mapping for saphana_sql2023_map: {'NVARCHAR': 'NVARCHAR', 'VARCHAR': 'VARCHAR', 'CHAR': 'CHAR', 'DATE': 'DATE', 'TIME': 'TIME', 'SECONDDATE': 'DATETIME2', 'TIMESTAMP': 'TIMESTAMP', 'DECIMAL': 'DECIMAL', 'SMALLINT': 'SMALLINT', 'INTEGER': 'INT', 'BIGINT': 'BIGINT', 'FLOAT': 'FLOAT', 'REAL': 'REAL', 'DOUBLE': 'FLOAT', 'CLOB': 'VARCHAR(max)', 'NCLOB': 'NVARCHAR(max)', 'BLOB': 'VARBINARY(max)', 'BOOLEAN': 'BIT', 'ST_GEOMETRY': 'JSON', 'ST_POINT': 'JSON', 'JSON': 'JSON', 'GUID': 'UNIQUEIDENTIFIER', 'TINYINT': 'SMALLINT'}
Processing file: sapmara_sql2023_map.json as database: sapmara_sql2023_map
Loaded mapping 

                                                    Version-5                                                               

Version 5-

->  Added stats compatibility between the actual data from the columns names 

In [43]:
import scipy.stats as stats
from scipy.stats import wasserstein_distance, ks_2samp
from sklearn.metrics import jaccard_score
from scipy.spatial.distance import euclidean
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import jaccard

In [44]:
df3 = pd.read_csv(r"C:/Users/Krithika.Patali/Downloads/synthetic_lhs_data.csv")
df4 = pd.read_csv(r"C:/Users/Krithika.Patali/Downloads/synthetic_rhs_data.csv")

#df3 = pd.read_csv(r"C:\Users\Yasvanth.Pamidi\OneDrive - ENCORA\Desktop\DataMap\synthetic data\synthetic_lhs_data.csv")
#df4 = pd.read_csv(r"C:\Users\Yasvanth.Pamidi\OneDrive - ENCORA\Desktop\DataMap\synthetic data\synthetic_rhs_data.csv")

In [46]:
def convert_numeric_values(series):
    try:
        return pd.to_numeric(series, errors='coerce')
    except Exception as e:
        print(f"Conversion error: {e}")
        return series

def compute_statistical_similarity(lhs_col, rhs_col,dtype_category):
    """
    Determines if two data distributions are statistically similar based on their type.
    """
    similarity_score = 0
    
    lhs_values = lhs_col.dropna()
    rhs_values = rhs_col.dropna()
    
    if dtype_category in ["INT", "INTEGER", "BIGINT", "SMALLINT", "FLOAT", "DOUBLE", "REAL", "NUMERIC", "DECIMAL"]:
        # Compute Wasserstein distance
        lhs_values = convert_numeric_values(lhs_col.dropna())
        rhs_values = convert_numeric_values(rhs_col.dropna())
        
        if len(lhs_values) > 0 and len(rhs_values) > 0:
            wasserstein_dist = stats.wasserstein_distance(lhs_values, rhs_values)
            ks_stat, ks_p_value = stats.ks_2samp(lhs_values, rhs_values)
            similarity_score = 1 / (1 + wasserstein_dist) if wasserstein_dist > 0 else 1
            similarity_score = min(similarity_score, 1 - ks_stat)  # Normalize
        else:
            similarity_score = 0  # No valid comparison
    
    elif dtype_category in ["CHAR", "VARCHAR", "TEXT", "STRING", "JSON"]:
        # Categorical columns - use Jaccard Similarity
        
        print("Jaccard Similarity")
        lhs_values = lhs_col.dropna().astype(str).unique()
        rhs_values = rhs_col.dropna().astype(str).unique()
        
        if len(lhs_values) > 0 and len(rhs_values) > 0:
            vectorizer = CountVectorizer(preprocessor=lambda x: x, binary=True)
            matrix = vectorizer.fit_transform([' '.join(lhs_values), ' '.join(rhs_values)])
            similarity_score = 1 - jaccard(matrix.toarray()[0], matrix.toarray()[1])
        else:
            print("something happen")
            similarity_score = 0  # No valid comparison
    else:
        print("NOT WORK FOR THIS DATA TYPE")
        similarity_score = 0  # Incompatible types
    
    return similarity_score
  
  
def truncate_percentage(value):
    return f"{value:.2f}"

In [54]:

def retrieve_top_similar_sentences_json(selected_index, top_n, normalized_similarity_matrix, df1, df2, column1, column2, type_mapping, compatibilities, filter_compatible=True):
    
    """
    Retrieves the top similar sentences based on a similarity matrix while checking compatibility.

    Args:
        selected_index (int): Index of the LHS column to compare.
        top_n (int): Number of top matches to return.
        normalized_similarity_matrix (np.array): Precomputed similarity scores.
        df1 (pd.DataFrame): LHS dataset.
        df2 (pd.DataFrame): RHS dataset.
        column1 (str): Column name for description in LHS.
        column2 (str): Column name for description in RHS.
        type_mapping (dict): Mapping of data types.
        compatibilities (dict): Compatibility mapping.
        similarity_threshold (float): Minimum similarity score for a match.
        filter_compatible (bool, optional): Whether to filter only compatible matches. Defaults to True.

    Returns:
        dict: JSON output with matched results.
    """

    if normalized_similarity_matrix.shape != (len(df1), len(df2)):

        print(f"Error: Similarity matrix shape {normalized_similarity_matrix.shape} does not match dataframes: LHS {len(df1)}, RHS {len(df2)}")
        raise ValueError("The similarity matrix dimensions must match the LHS and RHS datasets.")
    
    print("Input validation passed. Processing similarity matches...")

    similarities = normalized_similarity_matrix[selected_index]
    matches = []

    normalized_lhs_type = None
    stat_similarity = 0

    for idx in range(len(similarities)):
        similarity_score = float(similarities[idx])  # Extract similarity score
        if similarity_score < similarity_threshold:
            continue  # Skip if below threshold
        similarity_score_truncated = truncate_percentage(similarity_score * 100)

        rhs_field = df2.loc[idx, 'Attribute']
        rhs_desc = df2.loc[idx, column2]
        rhs_type = df2.loc[idx, 'Data_Type']
        rhs_length = df2.loc[idx, 'Length'] # RHS length

        print(f"\nProcessing RHS Index {idx}:")
        print(f"  Field: {rhs_field}, Description: {rhs_desc}, Type: {rhs_type}, Length: {rhs_length}")

        database_context = df2.loc[idx, 'Database'] if 'Database' in df2.columns else "default_database"
        normalized_rhs_type, database_context = normalize_data_type(rhs_type, None, type_mapping)

        print(f"  Normalized RHS Type: {normalized_rhs_type}, Database Context: {database_context}")

        lhs_type = df1.loc[selected_index, 'Data_Type'].upper()
        lhs_length = df1.loc[selected_index, 'Length']  # LHS length
        lhs_field = df1.loc[selected_index, 'Field Name']
        normalized_lhs_type, _ = normalize_data_type(lhs_type, None, type_mapping)

        print(f"  LHS Type: {lhs_type}, Normalized LHS Type: {normalized_lhs_type}")

        normalized_rhs_type = normalized_rhs_type.upper()
        is_compatible = are_compatible(lhs_type, normalized_rhs_type, compatibilities)

        # Normalize length before checking compatibility
        #lhs_length = normalize_length(lhs_length)
        #rhs_length = normalize_length(rhs_length)

        length_compatible = is_length_compatible(lhs_length, rhs_length, check_length=True)
        
        print(f"length_compatible: {length_compatible}")
        
        if lhs_field in df3.columns and rhs_field in df4.columns:
            print(f"Statistical Similarity: {lhs_field} vs {rhs_field}")
            lhs_col = df3[lhs_field]
            rhs_col = df4[rhs_field]
            
            if(is_compatible):
                stat_similarity = compute_statistical_similarity(lhs_col, rhs_col, normalized_rhs_type)
        else:
            stat_similarity = 0  # Incompatible types
        print(f"Statistical Similarity Score: {stat_similarity}")
        stat_similarity_percentage = truncate_percentage(stat_similarity * 100)

        
        if is_compatible and length_compatible and stat_similarity >= 0.3:
            matches.append({
                "rank": None,  # Placeholder to set rank later
                "similarity_score": similarity_score_truncated,
                "rhs_index": int(df2.index[idx]),
                "rhs_field_name": rhs_field,
                "rhs_field_desc": rhs_desc,
                "rhs_data_type": rhs_type,
                "normalized_rhs_type": normalized_rhs_type,
                "database_name": database_context if database_context else "Unknown",
                "rhs_length": convert_to_native_type(rhs_length),
                "compatibility": "Compatible",
                "statistical_similarity": stat_similarity_percentage  
            })

    # Sort matches by similarity score (descending)
    matches = sorted(matches, key=lambda x: x["similarity_score"], reverse=True)[:top_n]

    print("\nSorted Matches by Similarity Score:")

    for rank, match in enumerate(matches, start=1):
        match["rank"] = rank

    if not matches:
        matches.append({"message": "No compatible matches found."})

    # Fetch LHS details safely
    lhs_field = df1.loc[selected_index,'Field Name']
    lhs_desc = df1.loc[selected_index, column1]
    lhs_type = df1.loc[selected_index, 'Data_Type']
    lhs_length = df1.loc[selected_index, 'Length']   

    result = {
        "lhs_field_index": selected_index,
        "lhs_field_name": lhs_field,
        "lhs_field_description": lhs_desc,
        "lhs_field_data_type": lhs_type,
        "normalized_lhs_type": normalized_lhs_type,
        "lhs_field_length": convert_to_native_type(lhs_length),  
        "matches": matches
    }

    return {"results": [result]}

# Load type mappings and compatibilities
type_mapping = load_mappings(mappings_dir)
compatibilities_file = os.path.join(mappings_dir, 'compatibilities.json')
compatibilities = load_compatibilities(compatibilities_file)


# User Input with Validations

# Validate LHS Index
while True:
    try:
        enter_index = int(input("Enter the index from LHS to process: "))
        if 0<= enter_index <len(df1): # Ensure it's within range
            break
        else:
            print(f"Error: Index out of range. Please enter a value between 0 and {len(df1) - 1}.")
    except ValueError:
        print("Invalid input. Please enter a valid integer.")


# Validate Top N
while True:
    try:
        top_n = int(input("Enter the number of top similar sentences to retrieve (N): ") or 3)
        if top_n > 0:
            break
        else:
            print("Error: Please enter a positive integer greater than 0.")
    except ValueError:
        print("Invalid input. Please enter a valid integer.")

# Validate Similarity Score Threshold
while True:
    try:
        similarity_threshold = float(input("Enter the similarity score threshold (0.0 - 1.0): "))
        if 0.0 <= similarity_threshold <= 1.0:
            break
        else:
            print("Error: Please enter a value between 0.0 and 1.0.")
    except ValueError:
        print("Invalid input. Please enter a numeric value between 0.0 and 1.0.")

output_json = retrieve_top_similar_sentences_json(
    enter_index, top_n, normalized_similarity_matrix, df1, df2, "Description", "Description", type_mapping, compatibilities,similarity_threshold)
print(json.dumps(output_json, indent=2))

Loading mappings from directory: C:/Users/Krithika.Patali/DataMapper/Mappings
Processing file: pgsql_sql_2023_map.json as database: pgsql_sql_2023_map
Loaded mapping for pgsql_sql_2023_map: {'TEXT': 'VARCHAR', 'INTEGER': 'INT', 'VARCHAR': 'VARCHAR', 'FLOAT': 'FLOAT', 'BOOLEAN': 'BIT', 'BIGINT': 'BIGINT'}
Processing file: saphana_sql2023_map.json as database: saphana_sql2023_map
Loaded mapping for saphana_sql2023_map: {'NVARCHAR': 'NVARCHAR', 'VARCHAR': 'VARCHAR', 'CHAR': 'CHAR', 'DATE': 'DATE', 'TIME': 'TIME', 'SECONDDATE': 'DATETIME2', 'TIMESTAMP': 'TIMESTAMP', 'DECIMAL': 'DECIMAL', 'SMALLINT': 'SMALLINT', 'INTEGER': 'INT', 'BIGINT': 'BIGINT', 'FLOAT': 'FLOAT', 'REAL': 'REAL', 'DOUBLE': 'FLOAT', 'CLOB': 'VARCHAR(max)', 'NCLOB': 'NVARCHAR(max)', 'BLOB': 'VARBINARY(max)', 'BOOLEAN': 'BIT', 'ST_GEOMETRY': 'JSON', 'ST_POINT': 'JSON', 'JSON': 'JSON', 'GUID': 'UNIQUEIDENTIFIER', 'TINYINT': 'SMALLINT'}
Processing file: sapmara_sql2023_map.json as database: sapmara_sql2023_map
Loaded mapping 

In [55]:
lhs_field2 = output_json["results"][0]["lhs_field_name"]

In [56]:
rhs_fields = [match["rhs_field_name"] for match in output_json["results"][0]["matches"]]

In [57]:
import pandas as pd
import IPython.display as display

# File paths
lhs_file_path = r"C:\Users\Krithika.Patali\Downloads\synthetic_lhs_data.csv"
rhs_file_path = r"C:\Users\Krithika.Patali\Downloads\synthetic_rhs_data.csv"

# Load data into DataFrames
lhs_df = pd.read_csv(lhs_file_path)
rhs_df = pd.read_csv(rhs_file_path)

# Define columns of interest
lhs_column = output_json["results"][0]["lhs_field_name"]
rhs_columns = [match["rhs_field_name"] for match in output_json["results"][0]["matches"]]

# Extract relevant columns
lhs_data = lhs_df[[lhs_column]]
rhs_data = rhs_df[rhs_columns]

# Generate summary statistics
summary_stats = pd.DataFrame(columns=["Column", "Null Values", "Distinct Values", "Non-Distinct Values", "Most Common Value", "Mean Length", "Min Length", "Max Length"])

for col in [lhs_column] + rhs_columns:
    null_values = lhs_df[col].isnull().sum() if col == lhs_column else rhs_df[col].isnull().sum()
    distinct_values = lhs_df[col].nunique() if col == lhs_column else rhs_df[col].nunique()
    total_values = len(lhs_df[col]) if col == lhs_column else len(rhs_df[col])
    non_distinct_values = total_values - distinct_values
    most_common_value = lhs_df[col].mode()[0] if col == lhs_column else rhs_df[col].mode()[0]
    
    mean_length = lhs_df[col].astype(str).str.len().mean() if col == lhs_column else rhs_df[col].astype(str).str.len().mean()
    min_length = lhs_df[col].astype(str).str.len().min() if col == lhs_column else rhs_df[col].astype(str).str.len().min()
    max_length = lhs_df[col].astype(str).str.len().max() if col == lhs_column else rhs_df[col].astype(str).str.len().max()
    
    summary_stats.loc[len(summary_stats)] = [col, null_values, distinct_values, non_distinct_values, most_common_value, mean_length, min_length, max_length]

# Display summary statistics
display.display(summary_stats)

# Generate detailed view with inferred patterns and data types
detailed_view = pd.DataFrame(columns=["Column", "Data Type", "Top Patterns", "Inferred Data Domain"])

for col in [lhs_column] + rhs_columns:
    data_type = lhs_df[col].dtype if col == lhs_column else rhs_df[col].dtype
    inferred_patterns = lhs_df[col].astype(str).str.extract(r'([A-Za-z]+)')[0].dropna().unique() if col == lhs_column else rhs_df[col].astype(str).str.extract(r'([A-Za-z]+)')[0].dropna().unique()
    inferred_data_domain = "Categorical" if data_type == 'object' else "Numerical"
    
    detailed_view.loc[len(detailed_view)] = [col, data_type, inferred_patterns, inferred_data_domain]

# Display detailed view
display.display(detailed_view)


,Column,Null Values,Distinct Values,Non-Distinct Values,Most Common Value,Mean Length,Min Length,Max Length
0,KZNFM,0,736,52,Home on pick enough.,16.960660,6,20
1,Category name - SAP Article Category,0,1279,186,Southern inside.,16.845051,7,20
2,PPH Class,0,1266,199,Between true.,16.853242,7,20


,Column,Data Type,Top Patterns,Inferred Data Domain
0,KZNFM,object,"[Space, Accept, Strong, Machine, Should, Bad, ...",Categorical
1,Category name - SAP Article Category,object,"[Huge, Loss, Finish, Work, List, Once, Stop, C...",Categorical
2,PPH Class,object,"[Together, Kind, If, Full, Realize, Guy, Bed, ...",Categorical
